<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Epidemics 101</h1>
<h1>Or why your exponential fits of CoVID numbers are wrong</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from EpiModel import *

import watermark

%load_ext watermark

%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [9]:
# %watermark -n -v -m -g -iv
%watermark -n -v -m -iv


watermark 2.0.2
networkx  2.4
numpy     1.18.1
pandas    1.0.3
scipy     1.4.1
Mon Mar 30 2020 

CPython 3.7.7
IPython 7.13.0

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores  : 12
interpreter: 64bit


Load default figure style

In [ ]:
plt.style.use('./d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## SI Model

The first model we will look at is called the SI model and it only consists of two compartments (susceptible and infectious) and one transion between them

In [ ]:
beta = 0.2
SI = EpiModel()
SI.add_interaction('S', 'I', 'I', beta)

print(SI)

In other words, when a susceptiblem person encounters an infectious person, s/he aquires the infection with probability 0.2. We can integrate it easily. If in our initial population of 100,000 individuals we have 10 who are infected at time zero

In [ ]:
N = 100000
I0 = 10

SI.integrate(100, S=N-I0, I=I0)

And to plot it:

In [ ]:
ax = (SI.I/N).plot(label='Infectious', color=colors[1])
ax.set_xlabel('Time')
ax.set_ylabel('Population')
#ax.set_title('SI Model')
ax.legend()

This isn't very interesting: after a few steps everyone is infected!

## SIR Model

A more interesting and realistic model is the SIR model. It allows people to recover from the infection after some time, so now we have 3 compartments and 2 transitions

In [ ]:
beta = 0.2
mu = 0.1

SIR = EpiModel()
SIR.add_interaction('S', 'I', 'I', beta)
SIR.add_spontaneous('I', 'R', mu)

In [ ]:
print(SIR)

And the dynamics is more interesting as well:

In [ ]:
SIR.integrate(365, S=N-I0, I=I0, R=0)

And a quick visualization

In [ ]:
ax = SIR.plot('SIR Model', normed=True)
ax.legend(['Susceptible', 'Infectious', 'Recovered'])

The purple line is the number of currently infectious cases as a function of time. As we can see, not all of the population is infectious at the same time, and, in fact, only about $80\%$ of the population is ever infected, as shown by the green curve representing the fraction of recovered.

The typical bell curve you're likely to see (as asked to flatten) is simply the number of infectious individuals as a function of time:

In [ ]:
ax=(SIR.I/N).plot(label='Infectious', color=colors[1])
ax.set_xlabel('time')
ax.set_ylabel('Population')
ax.legend()

## Confirmed cases

If you've been paying attention to the news, the numbers of confirmed cases you've been seeing about CoVID19 correspond to, some fraction, $\phi$ of the total number of people that got infected up to that point. We can calculate that easily by simply seeing how many "healthy" people we lost as a function of time:

$$ Confirmed = \phi\left(N-S\right)$$

If, say, $\phi=10\%$ or $\phi=20\%$ of everyone who gets infected takes the test, then:

In [ ]:
phi = 0.1
ax = ((phi*(N-SIR.S))/N).plot(color=colors[3], linestyle='--', label=r'$10\%$ testing')

phi = 0.2
((phi*(N-SIR.S))/N).plot(ax=ax, color=colors[3], linestyle=':', label=r'$20\%$ testing')


ax.set_xlabel('Time')
ax.set_ylabel('Confirmed Cases')
ax.legend()

Naturaly, the higher the testing percentage, the higher the number of confirmed cases. Please note that here we are ignoring the number of healthy individuals that take the test and it comes back negative as we are only interested in the number of confirmed cases.

And the number of recovered cases follows a similar path, with a few days lag. Here we apply the same factor of $\phi$ to the number of recovered, since in principle, those would be the only ones we could observe recovering.

In [ ]:
phi = 0.1
ax = ((phi*(N-SIR.S))/N).plot(label='Confirmed', color=colors[3])
((phi*SIR.R)/N).plot(ax=ax, label='Recovered', color=colors[2])
ax.set_xlabel('Time')
ax.set_ylabel('Cases')
ax.legend()

For the sake of clarity I'm making the simplifying assumption that test are instantanous and happen as soon as the infection happens. This is __slightly__ unrealistic :) 

And indeed the numbers do start off growing exponentially as can be easily seen by plotting them on a log-linear scale.

In [ ]:
phi = 0.1
ax = (phi*(N-SIR.S[:50])).plot(color=colors[3], label='Confirmed')
t = np.arange(0, 50, 1)
y = 2*np.exp((beta-mu)*t)
ax.plot(t, y, lw=1, linestyle='--', label='Exponential Trend')

ax.set_yscale('log')
ax.set_xlabel('Time')
ax.set_ylabel('Cases')
ax.set_xlim(0, 50)
ax.set_ylim(1, 500)
ax.legend()

And we easily calculate the doubling time

In [ ]:
doubling_time = np.log(2)/(beta-mu)
print("Doubling time: %u days" % np.round(doubling_time))

## Non-uniform testing

Now let's look at a more realistic case. What if, instead of testing exactly $\phi=10\%$ of the cases, starting imediatly, it takes us a while to ramp up testing? Say, we start at 0 for the first week and smootly ramp up over the course of three weeks to a steady state rate of $\phi$?

In [ ]:
phi_t = np.ones(364)*0.1 # The steady state rate
phi_t[:7] = 0 # 0 for the first 7 days
phi_t[7:28] = np.linspace(0, 0.1, 21) # ramp up over 3 weeks

Now our curves looks a bit more interesting.

In [ ]:
ax = SIR.I.iloc[:49].plot(label='Real cases', color=colors[1])
(phi*(N-SIR.S)).iloc[:49].plot(ax=ax, label='Uniform testing', color=colors[3], linestyle='--')
(phi_t*(N-SIR.S)).iloc[:49].plot(ax=ax, label='Dynamic testing rate', color=colors[3])

t = np.arange(0, 49, 1)
y = 2*np.exp((beta-mu)*t)
ax.plot(t, y, lw=2, linestyle='--', label='Exponential Trend')
ax.set_ylim(1, 1000)
ax.legend()

ax.set_xlabel('Time')
ax.set_ylabel('Confirmed Cases')
ax.set_yscale('log')

Two things should be noted here:

- By the time we detect the first case, on day 11, the real number of cases is already several dozen
- The increase in testing rate gets muddled together with the increase in the number of cases to look like a much faster increase

Naturally, the opposite is also true, if we decrease the number of tests, we see an artificial slowing down of the number of cases

In [ ]:
phi_t[35:42]=np.linspace(0.1, 0.05, 7) # Gradually reduce the number of tests in the 5th week
phi_t[42:] *= 0.5 # stay at 0.05 for the rest of the time

ax = SIR.I.iloc[:49].plot(label='Real cases', color=colors[1])
(phi*(N-SIR.S)).iloc[:49].plot(ax=ax, label='Uniform testing', color=colors[3], linestyle='--')
(phi_t*(N-SIR.S)).iloc[:49].plot(ax=ax, label='Dynamic testing rate', color=colors[3])

t = np.arange(0, 49, 1)
y = 2*np.exp((beta-mu)*t)
ax.plot(t, y, lw=1, linestyle='--')
ax.set_ylim(1, 1000)
ax.legend()

ax.set_xlabel('Time')
ax.set_ylabel('Confirmed Cases')
ax.set_yscale('log')

In practice, the testing rate is not fixed and changes over time due to supply availability, policy changes, etc. As a result, trying to fit exponential curves to the numbers you hear in the news is at best, misleading.

## Dynamical lags

Another issue that can easily complicate things is the fact that epidemic models work a bit like conveyer belts. Susceptible get fed on one end, become infectious and eventually Recovered come out the other end. 

We calculate the number of __new infections__ as a function of time by looking at the change in the number of susceptibles and compare with the total number of currently infectious people. As the two curves have a very different range of values, we we normalize each curve by dividing it by its respective maximum value.

In [ ]:
new_infections = (-SIR.S).diff(1)

new_infections_max = new_infections.argmax()
infectious_max = SIR.I.argmax()

ax=(new_infections/new_infections.max()).plot(label='New Infections', color=colors[4])
(SIR.I/SIR.I.max()).plot(ax=ax, label='Currently Infectious', color=colors[1])
ax.vlines(x=[new_infections_max, infectious_max], ymin=0, ymax=1, lw=1, linestyle='--')
ax.set_ylabel('Fraction of maximum')
ax.set_xlabel('Time')
ax.legend()

As we can see, there's a clear lag between the point as the rate of new infectious starts decreasing and the number of currently infectious individuals starts decreasing as well.

## Lockdown

A consequence of this is that even if you completely cut the supply of Susceptibles who become Infected it takes some time for the effects to be seen. We illustrate this more clearly by implementing a simple lockdown strategy. Starting at time 75, we implement our lockdown strategy and completly stop new infections from occurring. We do this by replacing the epidemic model with one where people are only allowed to recover.

We start with the original SIR model as before and integrating for the first 75 days

In [ ]:
beta = 0.2
mu = 0.1

SIR2 = EpiModel()
SIR2.add_interaction('S', 'I', 'I', beta)
SIR2.add_spontaneous('I', 'R', mu)

SIR2.integrate(75, S=N-I0, I=I0, R=0)

Now we create new model with just one transition and setting the initial population to be the population at the end of the previous process

In [ ]:
Quarantine = EpiModel('SIR')
Quarantine.add_spontaneous('I', 'R', mu)

population = SIR2.values_.iloc[-1]
S0 = population.S
I0 = population.I
R0 = population.R

Quarantine.integrate(365-74, S=S0, I=I0, R=R0)

In [ ]:
Quarantine

Now we compbine the results from the two models

In [ ]:
values = pd.concat([SIR2.values_, Quarantine.values_], axis=0, ignore_index=True)

In [ ]:
ax = (values/N).plot()
ax.vlines(x=74, ymax=1, ymin=0, lw=1, linestyle='--')
ax.set_ylabel('Population')
ax.set_xlabel('Time')
ax.legend(['Susceptible', 'Infectious', 'Recovered'])

Even in this idealized scenario it still takes ~25 days before all the infectious individual recover and life can go back to normal.

If, instead, we have the more realistic scenario where instead of completely stopping the spread we reduce the the spreading $R_0$ to, say, 0.5 (the perfect scenario was equivalent to $R_0=0$, so that the epidemic becomes non viable, we have:

In [ ]:
beta = 0.2/4 # Reduce R0 by 4.
mu = 0.1

Quarantine2 = EpiModel()
Quarantine2.add_interaction('S', 'I', 'I', beta)
Quarantine2.add_spontaneous('I', 'R', mu)

Quarantine2.integrate(365-74, S=S0, I=I0, R=R0)

values2 = pd.concat([SIR2.values_, Quarantine2.values_], axis=0, ignore_index=True)

Now we can compare the two scenarios. Lighter dashed lines representing the perfect case described above

In [ ]:
ax = (values2/N).plot()
ax.vlines(x=74, ymax=1, ymin=0, lw=1, linestyle='--')
(values.S/N).plot(ax=ax, lw=2, linestyle='-', c=colors[0])
(values.I/N).plot(ax=ax, lw=2, linestyle='-', c=colors[1])
(values.R/N).plot(ax=ax, lw=2, linestyle='-', c=colors[2])
ax.set_ylabel('Population')
ax.set_xlabel('Time')
ax.legend(['Susceptible', 'Infectious', 'Recovered'])

Naturally, this requries the social distancing procedure to continue for longer and at the end we'll have more people who have been infected. However, if we stop too early, we simply go back to business as usual

In [ ]:
population = values2.iloc[100]
S0 = population.S
I0 = population.I
R0 = population.R


SIR2.integrate(365-99, S=S0, I=I0, R=R0)

values3 = values2.iloc[:100].copy()
values3 = pd.concat([values3, SIR2.values_], axis=0, ignore_index=True)

For comparison we now use the original SIR model withouth any intervention. 

In [ ]:
fig, ax = plt.subplots(1)

lines = (values3/N).plot(ax=ax)
ax.axvspan(xmin=74, xmax=100, alpha=0.3, color=colors[3])
(SIR.S/N).plot(ax=ax, lw=2, linestyle='--', c=colors[0], legend=False)
(SIR.I/N).plot(ax=ax, lw=2, linestyle='--', c=colors[1], legend=False)
(SIR.R/N).plot(ax=ax, lw=2, linestyle='--', c=colors[2], legend=False)

(values2.S/N).plot(ax=ax, lw=2, linestyle='-', c=colors[0])
(values2.I/N).plot(ax=ax, lw=2, linestyle='-', c=colors[1])
(values2.R/N).plot(ax=ax, lw=2, linestyle='-', c=colors[2])

ax.set_ylabel('Population')
ax.set_xlabel('Time')
ax.legend(['Susceptible', 'Infectious', 'Recovered'])

And zooming in on the number of Infectious

In [ ]:
fig, ax = plt.subplots(1)

lines = (values3.I/N).plot(ax=ax, c=colors[1])
span = ax.axvspan(xmin=74, xmax=100, alpha=0.3, color=colors[3])
(values2.I/N).plot(ax=ax, lw=2, linestyle='-', c=colors[1])
(SIR.I/N).plot(ax=ax, lw=2, linestyle='--', c=colors[1])


ax.set_ylabel('Population')
ax.set_xlabel('Time')
ax.legend(['Interrupted Quarantine', 'Continuous Quarantine', 'No Quarantine', ])

As you can see, even a broken social distancing procedure is better than nothing. It buys time and reduces the number of overall infected people in the population.

## Multiple populations

So far we have considered only a single population, but a usual, reality is much more complex. The fundamental assumption of compartmental models is that the population is well mixed: in effect, everyone is in contact with everyone. 

Naturally, this is not a very realistic assumption for anything larger than a small town. Let's see what happens if instead of a town we have several towns of the same size that get infected at different points in time.

As we already saw, the number of infectious individuals for a single population is, simply:

In [ ]:
ax=(SIR.I/N).plot(label='Infectious', color=colors[1])
ax.set_xlabel('time')
ax.set_ylabel('Population')
ax.legend()

Countries, States, Regions and Cities aren't single homogenous population but rather a set of several tightly connected sub populations with the total numbers being the combination of the numbers in each area. If, after starting in 1 population, the infection spreads to a second and third ones after, say, 7 and 14 days, we have:

In [ ]:
infections = (SIR.I).copy()

fig, ax = plt.subplots(1)
ax.plot(infections/N, color=colors[1], label='1 population', linestyle=':')
ax.plot((infections+infections.shift(31).fillna(0))/(2*N), color=colors[1], label='2 populations', linestyle='--')
ax.plot((infections+infections.shift(31).fillna(0)+infections.shift(61).fillna(0))/(3*N), color=colors[1], label='3 populations', linestyle='-')
ax.set_xlabel('time')
ax.set_ylabel('Infectious cases')
ax.legend()

Naturaly, the exact details of the connections each subpopulation will determine how impacted the shape of the curve will be,

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>